In [1]:
from serverlesskv.Modify_attention_layer.ours.custom_serverless_attn import LlamaForCausalLM
from transformers import AutoTokenizer
import torch
import copy
from transformers import AutoConfig

/root/anaconda3/envs/kvcache/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "/dataset/crosspipe/OriginModel/Llama-2-7b-chat-hf/"
config = AutoConfig.from_pretrained(model_path)
prompt_text = 'In a small, bustling cafe nestled in the heart of a vibrant city, a serendipitous event unfolded, leaving a lasting impression on all who witnessed it. As the patrons sat sipping their coffees and engaging in animated conversations, a talented street musician entered the cafe, carrying a weathered guitar and radiating an aura of creativity.'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

In [3]:
config

LlamaConfig {
  "_name_or_path": "/dataset/crosspipe/OriginModel/Llama-2-7b-chat-hf/",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "heavy_ratio": 0.1,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "recent_ratio": 0.1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.38.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [4]:
model = LlamaForCausalLM.from_pretrained(model_path,torch_dtype=torch.float16)


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.02s/it]


In [5]:
model.to(torch.device('cuda:2'))

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaServerless(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Li

In [6]:
input_ids = tokenizer(prompt_text, add_special_tokens=False, return_tensors='pt').input_ids.to(torch.device("cuda:2"))
input_ids
generate_ids = model.generate(input_ids, max_new_tokens=64, return_dict_in_generate=True)
sq=generate_ids.sequences
# kv=generate_ids.past_key_values
generate_ids

##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
##################################################
###############################

GenerateDecoderOnlyOutput(sequences=tensor([[  512,   263,  2319, 29892,   289,   504,  1847,  5777,  1725, 17763,
           839,   297,   278,  5192,   310,   263,   325,  4626,   424,  4272,
         29892,   263,   724,   355,   666,   277,   681,  1741, 20220,   287,
         29892, 10124,   263,  1833,   292, 17188,   373,   599,  1058, 16277,
           287,   372, 29889,  1094,   278,  2373, 12628,  3290,  1354,  3262,
          1009, 19785, 12712,   322,  3033,  6751,   297, 17524,  9678,   800,
         29892,   263,  5969, 14927, 11952,  4696,   713,  7802,   278,  5777,
          1725, 29892, 19436,   263, 14826,   287, 11210,   322, 17937,  1218,
           385,   263,  2002,   310,   907, 28157, 29889, 29871, 29906, 29900,
         29906, 29896, 29871, 29900, 29900, 29900, 29871, 29896, 29900, 29900,
         29900, 29900, 29900, 29871, 29896, 29900, 29900, 29900, 29900, 29900,
         29871, 29896, 29900, 29900, 29900, 29900, 29900, 29900, 29871, 29896,
         29900, 

In [7]:
result = tokenizer.decode(generate_ids.sequences[0], skip_special_tokens=True)
print(result)

In a small, bustling cafe nestled in the heart of a vibrant city, a serendipitous event unfolded, leaving a lasting impression on all who witnessed it. As the patrons sat sipping their coffees and engaging in animated conversations, a talented street musician entered the cafe, carrying a weathered guitar and radiating an aura of creativity. 2021 000 100000 100000 1000000 1000000 10000000 1000000 1000000
